# Notebook Example - Tabular

## Detecting tabular ML models vulnerabilities in WandB with Giskard
This example demonstrates how to efficiently scan two tabular ML models for hidden vulnerabilities using Giskard, log produced artifacts and interpret results within the WandB through just a few lines of code. The two tabular ML models used are:

| Model    | Description                                                            | Training data   |
|----------|------------------------------------------------------------------------|-----------------|
| `model1` | A simple sklearn `LogisticRegression` model trained only for 5 epochs. | Titanic dataset |
| `model2` | A simple sklearn `LogisticRegression` model trained for 100 epochs.    | Titanic dataset |

In [ ]:
import wandb

from giskard import Model, Dataset, demo, explain_with_shap, scan

model1, df = demo.titanic(max_iter=5)
model2, __ = demo.titanic(max_iter=100)  # Data are same.
models = {"titanic-max_iter=5": model1, "titanic-max_iter=100": model2}

wrapped_data = Dataset(df=df, 
                       target="Survived",
                       cat_columns=['Pclass', 'Sex', "SibSp", "Parch", "Embarked"])

for model_name, model in models.items():
    wrapped_model = Model(model=model.predict_proba,
                           model_type="classification",
                           feature_names=['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], 
                           classification_labels=model.classes_)
    
    # Log artifacts to the new WandB run.
    wrapped_data.to_wandb(name=model_name)
    
    shap_explanation_result = explain_with_shap(wrapped_model, wrapped_data)
    shap_explanation_result.to_wandb()
    
    scan_results = scan(wrapped_model, wrapped_data)
    scan_results.to_wandb()
    
    test_suite = scan_results.generate_test_suite()
    test_suite.run().to_wandb()

    # Finish a current run.
    wandb.finish()

After logging artifacts to runs, you can access the WandB UI via <http://localhost:8080>. By default, you will get to the WandB workspace, where you can compare runs. We can view, how logged artifacts are displayed:   
  
A dataset: 
<img src="../../assets/integrations/wandb/wandb-dataset.png">

SHAP bar plots for categorical features:
<img src="../../assets/integrations/wandb/wandb-categorical-chart.png">

SHAP scatter plots for numerical features:
<img src="../../assets/integrations/wandb/wandb-numerical-chart.png">

SHAP global feature importance plot:
<img src="../../assets/integrations/wandb/wandb-global-chart.png">

Giskard scan results:
<img src="../../assets/integrations/wandb/wandb-scan-result.png">

Giskard test-suite results. Only single run's table can be displayed simultaneously:
<img src="../../assets/integrations/wandb/wandb-test-suite-result.png">